In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from quantum_classes import *
from pauli_matrices import *

In [ ]:
# Create a quantum system with a single qubit
natural_freq = 10 * 10**9
driving_freq = 10 * 10**9
rabi_freq = 5 * 10**6

# CCD parameters
phi_0, epsilon_m, phase_freq, theta_m = 0, rabi_freq/4, rabi_freq, 0, 

# Integration parameters
tol = 10**-6
evaluation_points = 1000
evaluation_time = 5*10**-6


# Points at which to evaluate the solution for 2 qubits
initial_state_2q = np.array([0 + 0j, 1 + 0j, 0 + 0j, 0 + 0j])  

# Create a quantum system with two qubits
hamiltonian_2q = Quantum_Hamiltonian(rabi_freq, 2, np.array([natural_freq, natural_freq+10**6]))
system_2q = Quantum_System(hamiltonian_2q, 2)